In [3]:
import sys , os 
import numpy as np
import math as m
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

In [31]:
mesurePlot="/home/crct/CRCT/JupyterNotebook/gammaIndex/6E/new/validation/pdd6eMesure.csv"
#gatePlot="/home/crct/CRCT/JupyterNotebook/gammaIndex/6E/new/validation/profil6eGateW.csv"
gatePlot="/home/crct/CRCT/JupyterNotebook/gammaIndex/6E/new/validation/pdd6eGate.csv"

mesurePlotData=pd.read_csv(mesurePlot, sep=';')
gatePlotData=pd.read_csv(gatePlot, sep=',')


In [32]:
LinearInterpolation=True
#Point every (in mm)
#NewStep=0.1

Threshold=False
#Of Dmax
ThresholdValue=0.1


#Gamma Index criterion 
DTA=2 #mm
deltaDmax=0.02 #%


name='validation_profile_6e_22'

gammaIndex(mesurePlotData, gatePlotData, DTA, deltaDmax, name, Threshold)


RESULTAT GATE : 
*   Gamma Index Pass Rate 2.0% - 2mm : 99.0 %


In [29]:
def gammaIndex(mesurePlot, gatePlot, DTA, deltaDmax, name, Threshold=True):

    xGscale=0

    
    #Create numpy ndarray from pandas dataFrames
    refX=mesurePlotData['pas']
    refY=mesurePlotData['value']
    gateX=gatePlotData['pas']
    gateY=gatePlotData['value']

    


    # Compute Linear interpolation for simulated values and measured values according to your parameters (change 1000 values to change number of points)
    if LinearInterpolation==True:
        refX2=np.linspace(refX.min(), refX.max(), 2000)
        refY2=np.interp(refX2, refX, refY)
        gateX2=np.linspace(gateX.min(), gateX.max(), 2000)
        gateY2=np.interp(gateX2, gateX, gateY)
        
        
    else:
        refX2= refX
        refY2= refY
        gateX2= gateX
        gateY2= gateY
     
    #normalisation au centre 
    #maskFilm=(refX2 > -10) & (refX2 < 10)
    #maskGate=(gateX2 > -10) & (gateX2 < 10)
    
    
    #normalisation au max
    maxR=refY2.max()
    maxG=gateY2.max()
    #refY2=refY2/refY2[maskFilm].mean()
    #gateY2=gateY2/gateY2[maskGate].mean()
    

   
    
    refY2=refY2/maxR
    gateY2=gateY2/maxG
 
    #SCALING
    gateX2=gateX2+xGscale
    
    
    #Field size calculation:
      
    
    #create numpy ndarray for gamma comprisonj
    deltaD=np.zeros(len(refY2))
    gammaIndex=np.zeros(len(gateX2))
    gammaIndexTEMP=np.zeros(len(refX2))
    drTEMP=np.zeros(len(refX2))
    dr=np.zeros(len(refX2))

    #compute Gamma Index 1D
    for i in range(len(gateX2)):
        for j in range(len(refX2)):
            dr[j]=(gateX2[i]-refX2[j])
            deltaD[j]=gateY2[i]-refY2[j]
            gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/deltaDmax**2) + ((dr[j]**2)/DTA**2))           
        gammaIndex[i]=gammaIndexTEMP.min()

    #Compute Gamma Index Pass Rate (with threshold or not)
    nok=0.0
    a=0
    if Threshold==True: 
        for i in range(len(gammaIndex)):

            if gateY2[i]>0.10:
                a=a+1
                if gammaIndex[i]<1:
                    nok=nok+1
        print ('nombre de points testés : ' +str(a))
        print ('nombre de points passant le test (<1) : ' +str(nok))
        passRate=(nok/a)*100
    else:
        nok=(gammaIndex < 1).sum()
        passRate=nok*100.0/len(gammaIndex)
                                                                        
    #print results
    percentDmax=deltaDmax*100
    print('RESULTAT GATE : ')
    print('*   Gamma Index Pass Rate '+str(percentDmax) + '% - ' +str(DTA) + 'mm : '  +str(passRate) +' %')
    
    
    
    #compute diff % 
    diffGate=((refY2-gateY2)/refY2)*100

    
    # Plot profil or PDD with plotly      
    trace1 = go.Scatter(
        x = refX2,
        y = refY2,
        mode = 'line', 
        name = 'Measure', 
        line = dict(
            color = ('rgba(214, 39, 40, 100)')
        )
    )

    trace2 = go.Scatter(
        x = gateX2,
        y = gateY2,
        mode = 'line', 
        name = 'GATE', 
        line = dict(
            color = ('rgba(44, 160, 44, 100)'),
            dash='dot', 
            width=3
        )
    )
    
        #Plot gamma Index with plotly
    trace4 = go.Scatter(
        x = gateX2,
        y = gammaIndex,
        mode = 'lines', 
        name = 'Gamma Index GATE ', 
        line = dict(
            color = ('rgba(44, 160, 44, 100)'), 
            dash='dot'
        )
    )
     

    
    layout1= go.Layout(
        title='Profile Dose - 6 MeV',
        titlefont=dict(
        size=24),
        #annotations=[dict(
         #   x=28,
          #  y=0.5,
           # xref='refX2',
            #yref='refY2',
            #showarrow=False, 
            #text='Depth: 14mm',
            #font=dict(
            #size=16)
        #)],
        xaxis=dict(
            range=[0, 100],
            title='Off axis distance (mm)',
            titlefont=dict(
                size=20
            )
    ),
        
        yaxis=dict(
            title='Relative Dose',
            titlefont=dict(
                size=20)
        )
    )
    
        
    layout2= go.Layout(
        title='Gamma Index - 6 MeV',
        titlefont=dict(
        size=24),
        annotations=[dict(
            x=35,
            y=2,
            xref='gateX2',
            yref='gammaIndex',
            showarrow=False, 
            text='GIPR 2%-2mm: ' +str(passRate) + '%',
            font=dict(
            size=18)
         )],
        xaxis=dict(
            title='Depth (mm)',
            range=[0,70],
            titlefont=dict(
                    size=20)
        ),
        yaxis=dict(
            title='Gamma index value',
            range=[0, 3],
            titlefont=dict(
                    size=20)
        ),
        
    )
    
    
    layout3= go.Layout(
        xaxis=dict(
            title='Off distance axis (mm)',
            range=[0,70],
            titlefont=dict(
                size=20)
    ),
        yaxis=dict(
            title='Difference %',
            #range=[-30, 30],
            titlefont=dict(
                    size=20)
    )
    )
    
    
    trace6 = go.Scatter(
        x = gateX2,
        y = diffGate,
        mode = 'lines', 
        name = 'Difference % GATE ', 
        line = dict(
            color = ('rgba(44, 160, 44, 100)'), 
            dash='dash'
        )
    )
         

    
    data1 = [trace1, trace2]
    fig1= go.Figure(data=data1, layout=layout1)
    
    py.iplot(fig1, filename= name+'_plot', image='svg') 
    
    data2= [trace4]
    fig2= go.Figure(data=data2, layout=layout2)
    
    py.iplot(fig2, filename=name+'_gamma', image='png') 
    
    data3= [trace6]
    fig3= go.Figure(data=data3, layout=layout3)
    
    py.iplot(fig3, filename='Diff'+name, image='png') 

      
                                
    return;

